# Aggression Detection

https://sites.google.com/view/trac1/shared-task

## Imports

In [1]:
import csv
import nltk
import numpy as np
import time

from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from google.cloud import translate

## Parameters

In [6]:
MAX_VOCABULARY_SIZE = 20000
MAX_SEQ_LENGTH = 100
EMBEDDING_DIM = 16

EPOCHS = 10
BATCH_SIZE = 64

augmentation_languages = ['ger','fre','ru']

training_data = 'train/agr_en_train.csv'
validation_data = 'train/agr_en_dev.csv'
test_data = 'test/agr_en_fb_gold.csv'

## Constants

In [3]:
STOPWORDS = set(stopwords.words('english'))
categories = {'NAG': [1,0,0], 'CAG': [0,1,0], 'OAG': [0,0,1]}
translate_client = translate.Client()
wnl = nltk.WordNetLemmatizer()

## Define helper functions

In [4]:
def augument_and_save(row, language, filename):
    sentence = str(row[1].strip('\"').strip('\''))

    # Translate sentence to target language
    translation = translate_client.translate(
                sentence,
                source_language='eng',
                target_language=language)['translatedText']

    # Translate sentence from target language to english
    augmentation = translate_client.translate(
                translation,
                source_language=language,
                target_language='eng')['translatedText']

    # Save to disk
    with open('augmented/'+filename+"_"+language+".csv", 'a') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow([row[0], augmentation, row[2]])

    return augmentation

def preprocess_data(sentence):
    # Tokenize sentence
    tokens = nltk.word_tokenize(sentence)

    # Remove non-alphabetic characters
    tokens = [word.lower() for word in tokens if word.isalpha()]

    # Lemmatizations
    tokens = [wnl.lemmatize(t) for t in tokens]

    # Remove stop-words
    tokens = [word for word in tokens if word not in STOPWORDS]
    
    return tokens

## Augment data

In [5]:
with open(training_data) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:            
        filename = training_data.split("/")[-1].replace('.csv', '')

        for lang in augmentation_languages:
            attemps = 0
            while(attemps<3):
                try:
                    augument_and_save(row, lang, filename)
                    break
                except:
                    attemps += 1
                    print("Google Translate API req failed, sleeping for 1 min before trying again..")
                    time.sleep(60)
                    
            if attemps == 3:
                raise RuntimeError("Google Translate API request failed 3 times in a row.")


Google Translate API req failed, sleeping for 1 min before trying again..


## Import training, validation, and test data

In [7]:
train_X = []
train_y = []

val_X = []
val_y = []

test_X = []
test_y = []

training_paths = [training_data]

# Add paths to the augumented data
for language in augmentation_languages:
    filename = training_data.split("/")[-1].replace('.csv', '')
    training_paths.append('augmented/'+filename+'_'+language+'.csv')

# Import training data
for path in training_paths:
    with open(path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            sentence = str(row[1].strip('\"').strip('\''))

            train_X.append(preprocess_data(sentence))
            train_y.append(categories[str(row[2])])

# Import validation data
with open(validation_data) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            sentence = str(row[1].strip('\"').strip('\''))

            val_X.append(preprocess_data(sentence))
            val_y.append(categories[str(row[2])])
            
# Import test data
with open(test_data) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            sentence = str(row[1].strip('\"').strip('\''))

            test_X.append(preprocess_data(sentence))
            test_y.append(categories[str(row[2])])

## Vectorize dataset

In [8]:
tokenizer = Tokenizer(num_words=MAX_VOCABULARY_SIZE)
tokenizer.fit_on_texts(train_X)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 24662 unique tokens.


In [9]:
train_X_vec = pad_sequences(tokenizer.texts_to_sequences(train_X), maxlen=MAX_SEQ_LENGTH)
val_X_vec = pad_sequences(tokenizer.texts_to_sequences(val_X), maxlen=MAX_SEQ_LENGTH)
test_X_vec = pad_sequences(tokenizer.texts_to_sequences(test_X), maxlen=MAX_SEQ_LENGTH)

print('train_X_vec shape:', train_X_vec.shape)
print('val_X_vec shape:', val_X_vec.shape)
print('test_X_vec shape:', test_X_vec.shape)

train_X_vec shape: (47996, 100)
val_X_vec shape: (3001, 100)
test_X_vec shape: (916, 100)


## Convert to Numpy array

In [10]:
train_X_vec = np.array(train_X_vec)
train_y = np.array(train_y)

val_X_vec = np.array(val_X_vec)
val_y = np.array(val_y)

test_X_vec = np.array(test_X_vec)
test_y = np.array(test_y)

## Define model

In [11]:
model = Sequential()
model.add(Embedding(MAX_VOCABULARY_SIZE, EMBEDDING_DIM, input_length=MAX_SEQ_LENGTH))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(25, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Train model

In [12]:
history = model.fit(train_X_vec, 
                    train_y, 
                    validation_data=(val_X_vec, val_y), 
                    epochs=EPOCHS, 
                    batch_size = BATCH_SIZE)

Train on 47996 samples, validate on 3001 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
47996/47996 [==============================] - 88s 2ms/sample - loss: 0.9344 - acc: 0.5374 - val_loss: 0.9084 - val_acc: 0.5671
Epoch 2/10
47996/47996 [==============================] - 89s 2ms/sample - loss: 0.7870 - acc: 0.6429 - val_loss: 0.9224 - val_acc: 0.5778
Epoch 3/10
47996/47996 [==============================] - 87s 2ms/sample - loss: 0.7083 - acc: 0.6942 - val_loss: 0.9260 - val_acc: 0.5815
Epoch 4/10
47996/47996 [==============================] - 87s 2ms/sample - loss: 0.6552 - acc: 0.7244 - val_loss: 0.9534 - val_acc: 0.5795
Epoch 5/10
47996/47996 [==============================] - 87s 2ms/sample - loss: 0.6125 - acc: 0.7474 - val_loss: 1.0105 - val_acc: 0.5755
Epoch 6/10
 4608/47996 [=>............................] - ETA: 1:19 - loss: 0.5826 - acc: 0.7630

KeyboardInterrupt: 

## Test model

In [13]:
accuracy = model.evaluate(x=test_X_vec, y=test_y)

916/916 [==============================] - 1s 814us/sample - loss: 1.1090 - acc: 0.5557
